#### Imports

In [4]:
import pandas as pd
import numpy as np
import os

### Accidents

In [17]:
file_path_1 = 'data/accidents_1.csv'
file_path_2 = 'data/accidents_2.csv'

df1 = pd.read_csv(file_path_1)
df2 = pd.read_csv(file_path_2)

In [611]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33513 entries, 0 to 33512
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   fecha                   33513 non-null  object
 1   hora                    33513 non-null  object
 2   tipo_de_accidente       33513 non-null  object
 3   nombre_de_asentamiento  33513 non-null  object
 4   nombre_de_la_vialidad   33513 non-null  object
 5   georreferencia          33513 non-null  object
dtypes: object(6)
memory usage: 1.5+ MB


In [613]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33666 entries, 0 to 33665
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   urbana          33666 non-null  object
 1   tipaccid        33666 non-null  object
 2   automovil       33666 non-null  int64 
 3   causaacci       33666 non-null  object
 4   sexo            33666 non-null  object
 5   aliento         33666 non-null  object
 6   id_edad         33666 non-null  int64 
 7   clasacc         33666 non-null  object
 8   casualties_kil  33666 non-null  int64 
 9   casualties_inj  33666 non-null  int64 
 10  time            33666 non-null  object
 11  heavy_vehicles  33666 non-null  int64 
 12  light_vehicles  33666 non-null  int64 
 13  two_wheelers    33666 non-null  int64 
 14  other_vehicles  33666 non-null  int64 
 15  date            33666 non-null  object
dtypes: int64(8), object(8)
memory usage: 4.1+ MB


In [615]:
no_hour_df1 = df1[(df1.hora == 'sd') | (df1.hora == 'SD') | (df1.hora == 'No Dato')]
no_hour_df1

,fecha,hora,tipo_de_accidente,nombre_de_asentamiento,nombre_de_la_vialidad,georreferencia
582,2023-01-24,No Dato,Crucero,Centro,Diego de Montemayor,"25.6759721, -100.3046747"
586,2023-01-24,No Dato,Otro,La Republica,Plan de Paracuaro,"25.6246939, -100.2969558"
594,2023-01-24,No Dato,Alcance,Fracc Gonzalitos,Gonzalitos,"25.6940699, -100.3512487"
595,2023-01-24,No Dato,Alcance,Leones,Leones,"25.6968778, -100.3508409"
596,2023-01-24,No Dato,Lateral,Centro,Padre Mier,"25.6695782, -100.3216168"
...,...,...,...,...,...,...
33266,2023-12-20,sd,Lateral,Centro,Pino Suarez,"25.674989, -100.31919"
33336,2023-12-23,sd,Estrellamiento,Centro,Constitucion,"25.668089, -100.299054"
33358,2023-12-23,sd,Alcance,San Jose,Bernardo Reyes,"25.726136, -100.33149"
33373,2023-12-24,sd,Crucero,Independencia,San Luis Potosi,"25.6600142, -100.31241"


In [617]:
df1 = df1.rename(columns={
    "hora": "time",
    "fecha": "date"
})

In [619]:
values_to_remove = ['sd', 'no dato']
df1 = df1[~df1.time.str.lower().isin(values_to_remove)]

In [621]:
#Date
df1['date'] = pd.to_datetime(df1['date'])
df2['date'] = pd.to_datetime(df2['date'])

#Time
df1['time'] = pd.to_datetime(df1['time'], format='mixed').dt.time
df2['time'] = pd.to_datetime(df2['time'], format='mixed').dt.time

In [622]:
# Find duplicates based on two columns
duplicates = df1[df1.duplicated(subset=['date', 'time'], keep=False)]
duplicates

,date,time,tipo_de_accidente,nombre_de_asentamiento,nombre_de_la_vialidad,georreferencia
35,2023-01-05,09:00:00,Crucero,Cementos,Ruiz Cortines,"25.704591, -100.3296626"
53,2023-01-06,20:20:00,Lateral,Centro,Felix U Gomez,"25.6876835, -100.2962458"
62,2023-01-06,19:07:00,Lateral,San Jeronimo,Diaz Ordaz,"25.6744918, -100.3673184"
64,2023-01-06,11:13:00,Alcance,Cumbres San Angel,Lincoln,"25.7800991, -100.4548858"
67,2023-01-06,11:13:00,Lateral,Del Prado,Alfonso Reyes,"25.6990437, -100.3169253"
...,...,...,...,...,...,...
33478,2023-12-29,12:45:00,Crucero,Centro,Cuauhtemoc,"25.681841, -100.31753"
33487,2023-12-30,19:37:00,Alcance,Treviño,Guerrero,"25.69167, -100.311396"
33488,2023-12-30,19:19:00,Crucero,Ferrocarrilera,Palacio de Justicia,"25.691091, -100.338801"
33497,2023-12-30,19:37:00,Otro,Vidriera,Ruiz Cortines,"25.704228, -100.304438"


In [625]:
# Remove duplicates 
df1 = df1.drop(index=duplicates.index)

In [627]:
# Find duplicates based on two columns
duplicates2 = df2[df2.duplicated(subset=['date', 'time'], keep=False)]
duplicates2

,urbana,tipaccid,automovil,causaacci,sexo,aliento,id_edad,clasacc,casualties_kil,casualties_inj,time,heavy_vehicles,light_vehicles,two_wheelers,other_vehicles,date
82,Accidente en intersección,Colisión con vehículo automotor,2,Conductor,Hombre,No,25,Sólo daños,0,0,19:20:00,0,2,0,0,2023-01-02
83,Accidente en intersección,Colisión con vehículo automotor,2,Conductor,Hombre,No,76,Sólo daños,0,0,19:20:00,0,2,0,0,2023-01-02
84,Accidente en intersección,Colisión con peatón (atropellamiento),1,Conductor,Se fugó,Se ignora,0,No fatal,0,1,19:20:00,0,1,0,0,2023-01-02
86,Accidente en intersección,Colisión con vehículo automotor,2,Conductor,Hombre,No,99,Sólo daños,0,0,19:35:00,0,2,0,0,2023-01-02
87,Accidente en intersección,Colisión con vehículo automotor,2,Conductor,Hombre,No,64,Sólo daños,0,0,19:35:00,0,2,0,0,2023-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33598,Accidente en intersección,Colisión con motocicleta,1,Conductor,Hombre,No,99,Sólo daños,0,0,14:38:00,0,1,1,0,2023-12-30
33614,Accidente en intersección,Colisión con vehículo automotor,2,Conductor,Hombre,No,99,Sólo daños,0,0,19:19:00,0,2,0,0,2023-12-30
33615,Accidente en intersección,Colisión con objeto fijo,1,Conductor,Se fugó,Se ignora,0,Sólo daños,0,0,19:19:00,0,1,0,0,2023-12-30
33618,Accidente en intersección,Colisión con vehículo automotor,2,Conductor,Hombre,No,99,Sólo daños,0,0,19:37:00,0,2,0,0,2023-12-30


In [629]:
df2 = df2.drop(index=duplicates2.index)

#### Joining datasets

In [632]:
merged = pd.merge(df1, df2, on=['date', 'time'], how='inner', indicator=True)
merged = pd.merge(df1, df2, on=['date', 'time'], how='outer', indicator=True)

In [634]:
unmatched_df1 = merged[merged['_merge'] == 'left_only']
unmatched_df2 = merged[merged['_merge'] == 'right_only']

In [636]:
unmatched_df1

,date,time,tipo_de_accidente,nombre_de_asentamiento,nombre_de_la_vialidad,georreferencia,urbana,tipaccid,automovil,causaacci,...,aliento,id_edad,clasacc,casualties_kil,casualties_inj,heavy_vehicles,light_vehicles,two_wheelers,other_vehicles,_merge
129,2023-01-03,15:55:00,Alcance,Pio X,Morones Prieto,"25.6652002, -100.3307452",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1341,2023-01-20,00:01:00,Lateral,Centro,Cuauhtemoc,"25.6757314, -100.3183984",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1700,2023-01-24,20:20:00,Estrellamiento,Cumbres 5o Sector,Leones,"25.6880819, -100.365532",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1710,2023-01-25,00:01:00,Lateral,Martinez,Jose A Conchello,"25.6888586, -100.2945651",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1950,2023-01-28,00:01:00,Volcadura,Fierro,Constitucion,"25.6812873, -100.2729957",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2204,2023-01-31,10:10:00,Alcance,Del Norte,Dr Mora,"25.7051571, -100.3095645",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3480,2023-02-16,16:35:00,Alcance,Contry,Garza Sada,"25.6318283, -100.2799646",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3583,2023-02-17,17:20:00,Lateral,Rincon de la Primavera,Revolucion,"25.6493271, -100.27495",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4568,2023-03-01,19:40:00,Alcance,Cumbres del Sol,Leones,"25.7503835, -100.4302353",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4573,2023-03-01,20:10:00,Alcance,Mirasol,Lincoln,"25.765046309195974, -100.39825486131484",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [638]:
unmatched_df2

,date,time,tipo_de_accidente,nombre_de_asentamiento,nombre_de_la_vialidad,georreferencia,urbana,tipaccid,automovil,causaacci,...,aliento,id_edad,clasacc,casualties_kil,casualties_inj,heavy_vehicles,light_vehicles,two_wheelers,other_vehicles,_merge
0,2023-01-01,01:18:00,NaN,NaN,NaN,NaN,Accidente en intersección,Colisión con vehículo automotor,2.0,Conductor,...,Se ignora,0.0,Sólo daños,0.0,0.0,0.0,2.0,0.0,0.0,right_only
1,2023-01-01,04:44:00,NaN,NaN,NaN,NaN,Accidente en intersección,Incendio,1.0,Conductor,...,Se ignora,0.0,Sólo daños,0.0,0.0,0.0,1.0,0.0,0.0,right_only
2,2023-01-01,07:17:00,NaN,NaN,NaN,NaN,Accidente en intersección,Colisión con vehículo automotor,2.0,Conductor,...,No,62.0,Sólo daños,0.0,0.0,0.0,2.0,0.0,0.0,right_only
3,2023-01-01,08:06:00,NaN,NaN,NaN,NaN,Accidente en intersección,Colisión con objeto fijo,1.0,Conductor,...,No,99.0,Sólo daños,0.0,0.0,0.0,1.0,0.0,0.0,right_only
4,2023-01-01,10:10:00,NaN,NaN,NaN,NaN,Accidente en intersección,Colisión con objeto fijo,1.0,Conductor,...,No,69.0,Sólo daños,0.0,0.0,0.0,1.0,0.0,0.0,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29288,2023-12-23,22:30:00,NaN,NaN,NaN,NaN,Accidente en intersección,Colisión con objeto fijo,1.0,Conductor,...,No,99.0,Sólo daños,0.0,0.0,0.0,1.0,0.0,0.0,right_only
29297,2023-12-24,00:50:00,NaN,NaN,NaN,NaN,Accidente en intersección,Colisión con ciclista,0.0,Conductor,...,Se ignora,0.0,Sólo daños,0.0,0.0,0.0,0.0,1.0,1.0,right_only
29371,2023-12-26,01:13:00,NaN,NaN,NaN,NaN,Accidente en intersección,Colisión con objeto fijo,0.0,Conductor,...,Se ignora,0.0,Sólo daños,0.0,0.0,1.0,0.0,0.0,0.0,right_only
29478,2023-12-28,02:15:00,NaN,NaN,NaN,NaN,Accidente en intersección,Colisión con vehículo automotor,2.0,Conductor,...,No,99.0,Sólo daños,0.0,0.0,0.0,2.0,0.0,0.0,right_only


In [640]:
accidents_df = merged[merged['_merge'] == 'both']
accidents_df = accidents_df.reset_index()
accidents_df

,index,date,time,tipo_de_accidente,nombre_de_asentamiento,nombre_de_la_vialidad,georreferencia,urbana,tipaccid,automovil,...,aliento,id_edad,clasacc,casualties_kil,casualties_inj,heavy_vehicles,light_vehicles,two_wheelers,other_vehicles,_merge
0,92,2023-01-03,03:34:00,Estrellamiento,Burocratas del Estado,Rangel Frias,"25.7101678, -100.3655685",Accidente en intersección,Colisión con objeto fijo,1.0,...,Se ignora,0.0,Sólo daños,0.0,0.0,0.0,1.0,0.0,0.0,both
1,94,2023-01-03,06:27:00,Crucero,Tecnologico,Rio Panuco,"25.6477155, -100.2956425",Accidente en intersección,Colisión con ciclista,0.0,...,No,30.0,No fatal,0.0,1.0,1.0,0.0,1.0,0.0,both
2,116,2023-01-03,13:11:00,Estrellamiento,Chepe Vera,Madero,"25.6832535, -100.3427724",Accidente en intersección,Colisión con objeto fijo,1.0,...,No,99.0,Sólo daños,0.0,0.0,0.0,1.0,0.0,0.0,both
3,119,2023-01-03,14:24:00,Estrellamiento,Del Norte,Ladron de Guevara,"25.7123781, -100.303487",Accidente en intersección,Colisión con objeto fijo,0.0,...,Se ignora,0.0,Sólo daños,0.0,0.0,1.0,0.0,0.0,0.0,both
4,120,2023-01-03,14:27:00,Crucero,Barrio Moderna,Luis D Colosio,"25.7721352, -100.3890336",Accidente en intersección,Colisión con motocicleta,1.0,...,No,99.0,Sólo daños,0.0,0.0,0.0,1.0,1.0,0.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29215,29688,2023-12-31,20:43:00,Otro,Los Altos,San Juan de los Lagos,"25.716162, -100.36801",Accidente en intersección,Colisión con vehículo automotor,2.0,...,No,43.0,Sólo daños,0.0,0.0,0.0,2.0,0.0,0.0,both
29216,29689,2023-12-31,20:56:00,Crucero,Barrio Topo Chico,Luis D Colosio,"25.7484909, -100.387697",Accidente en intersección,Colisión con vehículo automotor,2.0,...,No,18.0,No fatal,0.0,1.0,0.0,2.0,0.0,0.0,both
29217,29690,2023-12-31,21:45:00,Alcance,Unidad Modelo,Aztlan,"25.740806, -100.353714",Accidente en intersección,Colisión con motocicleta,1.0,...,No,20.0,No fatal,0.0,2.0,0.0,1.0,2.0,0.0,both
29218,29691,2023-12-31,22:07:00,Crucero,Cumbres Allegro,Leones,"25.750282, -100.430029",Accidente en intersección,Colisión con vehículo automotor,2.0,...,No,99.0,Sólo daños,0.0,0.0,0.0,2.0,0.0,0.0,both


### Weather

In [19]:
file_path_3 = 'data/weather_mty_2023-01-01_to_2023-12-31.csv'
weather_df = pd.read_csv(file_path_3)

In [928]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              365 non-null    object 
 1   datetime          365 non-null    object 
 2   tempmax           365 non-null    float64
 3   tempmin           365 non-null    float64
 4   temp              365 non-null    float64
 5   feelslikemax      365 non-null    float64
 6   feelslikemin      365 non-null    float64
 7   feelslike         365 non-null    float64
 8   dew               365 non-null    float64
 9   humidity          365 non-null    float64
 10  precip            365 non-null    float64
 11  precipprob        365 non-null    int64  
 12  precipcover       365 non-null    float64
 13  preciptype        116 non-null    object 
 14  snow              365 non-null    int64  
 15  snowdepth         365 non-null    int64  
 16  windgust          365 non-null    float64
 1

In [930]:
weather_df = weather_df.drop(['name', 'tempmax', 'tempmin','preciptype', 'snow', 'snowdepth', 'sunrise', 'sunset',
                              'solarenergy', 'uvindex', 'solarradiation', 'sealevelpressure', 'dew', 'feelslikemax', 
                              'feelslikemin', 'conditions', 'moonphase', 'description', 'icon', 'stations'], axis=1)

In [932]:
weather_df = weather_df.rename(columns={
    "datetime": "date",
})

In [934]:
weather_df['date'] = pd.to_datetime(weather_df['date'])

In [936]:
np.array(weather_df.columns)

array(['date', 'temp', 'feelslike', 'humidity', 'precip', 'precipprob',
       'precipcover', 'windgust', 'windspeed', 'winddir', 'cloudcover',
       'visibility', 'severerisk'], dtype=object)

### Final Merge

In [939]:
df = pd.merge(accidents_df, weather_df, on='date', how='left')

In [941]:
df

,index,date,time,tipo_de_accidente,nombre_de_asentamiento,nombre_de_la_vialidad,georreferencia,urbana,tipaccid,automovil,...,humidity,precip,precipprob,precipcover,windgust,windspeed,winddir,cloudcover,visibility,severerisk
0,92,2023-01-03,03:34:00,Estrellamiento,Burocratas del Estado,Rangel Frias,"25.7101678, -100.3655685",Accidente en intersección,Colisión con objeto fijo,1.0,...,35.9,0.0,0,0.0,32.0,19.4,289.3,9.1,15.1,10
1,94,2023-01-03,06:27:00,Crucero,Tecnologico,Rio Panuco,"25.6477155, -100.2956425",Accidente en intersección,Colisión con ciclista,0.0,...,35.9,0.0,0,0.0,32.0,19.4,289.3,9.1,15.1,10
2,116,2023-01-03,13:11:00,Estrellamiento,Chepe Vera,Madero,"25.6832535, -100.3427724",Accidente en intersección,Colisión con objeto fijo,1.0,...,35.9,0.0,0,0.0,32.0,19.4,289.3,9.1,15.1,10
3,119,2023-01-03,14:24:00,Estrellamiento,Del Norte,Ladron de Guevara,"25.7123781, -100.303487",Accidente en intersección,Colisión con objeto fijo,0.0,...,35.9,0.0,0,0.0,32.0,19.4,289.3,9.1,15.1,10
4,120,2023-01-03,14:27:00,Crucero,Barrio Moderna,Luis D Colosio,"25.7721352, -100.3890336",Accidente en intersección,Colisión con motocicleta,1.0,...,35.9,0.0,0,0.0,32.0,19.4,289.3,9.1,15.1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29215,29688,2023-12-31,20:43:00,Otro,Los Altos,San Juan de los Lagos,"25.716162, -100.36801",Accidente en intersección,Colisión con vehículo automotor,2.0,...,54.1,0.0,0,0.0,20.2,12.8,222.1,16.8,13.1,10
29216,29689,2023-12-31,20:56:00,Crucero,Barrio Topo Chico,Luis D Colosio,"25.7484909, -100.387697",Accidente en intersección,Colisión con vehículo automotor,2.0,...,54.1,0.0,0,0.0,20.2,12.8,222.1,16.8,13.1,10
29217,29690,2023-12-31,21:45:00,Alcance,Unidad Modelo,Aztlan,"25.740806, -100.353714",Accidente en intersección,Colisión con motocicleta,1.0,...,54.1,0.0,0,0.0,20.2,12.8,222.1,16.8,13.1,10
29218,29691,2023-12-31,22:07:00,Crucero,Cumbres Allegro,Leones,"25.750282, -100.430029",Accidente en intersección,Colisión con vehículo automotor,2.0,...,54.1,0.0,0,0.0,20.2,12.8,222.1,16.8,13.1,10


### Data Cleaning and Preparation

In [944]:
df.columns

Index(['index', 'date', 'time', 'tipo_de_accidente', 'nombre_de_asentamiento',
       'nombre_de_la_vialidad', 'georreferencia', 'urbana', 'tipaccid',
       'automovil', 'causaacci', 'sexo', 'aliento', 'id_edad', 'clasacc',
       'casualties_kil', 'casualties_inj', 'heavy_vehicles', 'light_vehicles',
       'two_wheelers', 'other_vehicles', '_merge', 'temp', 'feelslike',
       'humidity', 'precip', 'precipprob', 'precipcover', 'windgust',
       'windspeed', 'winddir', 'cloudcover', 'visibility', 'severerisk'],
      dtype='object')

In [946]:
df.sexo.value_counts()

sexo
Hombre     20158
Mujer       6666
Se fugó     2396
Name: count, dtype: int64

In [948]:
df.aliento.value_counts()

aliento
No           26721
Se ignora     2396
Sí             103
Name: count, dtype: int64

In [950]:
# Replace "se fugó" (ran out) based on proportions
df.replace("Se fugó", pd.NA, inplace=True)

# Calculate proportions for each category
counts = df["sexo"].value_counts(normalize=True, dropna=True)
categories = counts.index
proportions = counts.values

def assign_values(row, categories, proportions):
    if pd.isna(row):
        return np.random.choice(categories, p=proportions) #randomize values
    return row

df['sex'] = df["sexo"].apply(lambda x: assign_values(x, categories, proportions))

In [952]:
# Replace "se ignora" (ignored) with mode
df.replace("Se ignora", pd.NA, inplace=True)
mode_value = df['aliento'].mode().iloc[0]  # Get the mode value of the column
df['alcohol'] = df['aliento'].fillna(mode_value)

In [954]:
df.columns

Index(['index', 'date', 'time', 'tipo_de_accidente', 'nombre_de_asentamiento',
       'nombre_de_la_vialidad', 'georreferencia', 'urbana', 'tipaccid',
       'automovil', 'causaacci', 'sexo', 'aliento', 'id_edad', 'clasacc',
       'casualties_kil', 'casualties_inj', 'heavy_vehicles', 'light_vehicles',
       'two_wheelers', 'other_vehicles', '_merge', 'temp', 'feelslike',
       'humidity', 'precip', 'precipprob', 'precipcover', 'windgust',
       'windspeed', 'winddir', 'cloudcover', 'visibility', 'severerisk', 'sex',
       'alcohol'],
      dtype='object')

In [956]:
df.tipo_de_accidente.value_counts()

tipo_de_accidente
Alcance              8808
Lateral              8744
Crucero              5061
Estrellamiento       2063
Otro                 2002
De Reversa           1218
Atropello             656
De Frente             290
Volcadura             236
Caida de Persona       62
Incendio               56
Caida de persona       22
Salida del Camino       2
Name: count, dtype: int64

In [958]:
df.tipaccid.value_counts()

tipaccid
Colisión con vehículo automotor          23556
Colisión con objeto fijo                  2420
Colisión con motocicleta                  1964
Colisión con peatón (atropellamiento)      658
Volcadura                                  382
Caída de pasajero                           84
Colisión con ciclista                       64
Incendio                                    56
Otro                                        25
Salida del camino                            7
Colisión con animal                          4
Name: count, dtype: int64

In [960]:
df.tipo_de_accidente.value_counts()

tipo_de_accidente
Alcance              8808
Lateral              8744
Crucero              5061
Estrellamiento       2063
Otro                 2002
De Reversa           1218
Atropello             656
De Frente             290
Volcadura             236
Caida de Persona       62
Incendio               56
Caida de persona       22
Salida del Camino       2
Name: count, dtype: int64

In [962]:
df['accident_type'] = df.apply(
    lambda row: row['tipo_de_accidente'] if row['tipaccid'] == 'Colisión con vehículo automotor' 
    else row['tipaccid'],
    axis=1
)

In [964]:
df.accident_type.value_counts()

accident_type
Alcance                                  8517
Lateral                                  8010
Crucero                                  4273
Colisión con objeto fijo                 2420
Colisión con motocicleta                 1964
Otro                                     1376
De Reversa                               1190
Colisión con peatón (atropellamiento)     658
Volcadura                                 382
De Frente                                 215
Caída de pasajero                          84
Colisión con ciclista                      64
Incendio                                   56
Salida del camino                           7
Colisión con animal                         4
Name: count, dtype: int64

In [966]:
df['month'] = pd.to_datetime(df['date']).dt.month
df['weekday'] = pd.to_datetime(df['date']).dt.dayofweek

In [968]:
df = df[['accident_type', 'nombre_de_asentamiento', 'nombre_de_la_vialidad', 'causaacci', 'sex', 'alcohol',
        'month', 'weekday', 'id_edad', 'casualties_kil', 'casualties_inj', 'heavy_vehicles', 'light_vehicles', 
        'two_wheelers', 'other_vehicles', 'precip', 'windgust', 'visibility', 'severerisk', 'clasacc']]

In [970]:
categorical = ['accident_type', 'nombre_de_asentamiento', 'nombre_de_la_vialidad', 'causaacci', 'sex', 'alcohol']
numerical = ['month', 'weekday', 'id_edad', 'casualties_kil', 'casualties_inj', 'heavy_vehicles', 'light_vehicles', 'two_wheelers', 
             'other_vehicles', 'precip', 'windgust', 'visibility', 'severerisk']
target = ['clasacc']

In [972]:
for c in categorical:
    df[c] = df[c].str.lower().str.replace(" ", "_")

/var/folders/x3/ksb10fdj52b1btlf9x9_nkx00000gn/T/ipykernel_28099/2007638387.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[c] = df[c].str.lower().str.replace(" ", "_")


In [974]:
class_mapping = {'Sólo daños': 0, 'No fatal': 1, 'Fatal': 2}
df['clasacc'] = df['clasacc'].map(class_mapping)

/var/folders/x3/ksb10fdj52b1btlf9x9_nkx00000gn/T/ipykernel_28099/2189893041.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clasacc'] = df['clasacc'].map(class_mapping)


In [976]:
df

,accident_type,nombre_de_asentamiento,nombre_de_la_vialidad,causaacci,sex,alcohol,month,weekday,id_edad,casualties_kil,casualties_inj,heavy_vehicles,light_vehicles,two_wheelers,other_vehicles,precip,windgust,visibility,severerisk,clasacc
0,colisión_con_objeto_fijo,burocratas_del_estado,rangel_frias,conductor,hombre,no,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,32.0,15.1,10,0
1,colisión_con_ciclista,tecnologico,rio_panuco,conductor,mujer,no,1,1,30.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,32.0,15.1,10,1
2,colisión_con_objeto_fijo,chepe_vera,madero,conductor,hombre,no,1,1,99.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,32.0,15.1,10,0
3,colisión_con_objeto_fijo,del_norte,ladron_de_guevara,conductor,hombre,no,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,32.0,15.1,10,0
4,colisión_con_motocicleta,barrio_moderna,luis_d_colosio,conductor,mujer,no,1,1,99.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,32.0,15.1,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29215,otro,los_altos,san_juan_de_los_lagos,conductor,hombre,no,12,6,43.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,20.2,13.1,10,0
29216,crucero,barrio_topo_chico,luis_d_colosio,conductor,hombre,no,12,6,18.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,20.2,13.1,10,1
29217,colisión_con_motocicleta,unidad_modelo,aztlan,conductor,hombre,no,12,6,20.0,0.0,2.0,0.0,1.0,2.0,0.0,0.0,20.2,13.1,10,1
29218,crucero,cumbres_allegro,leones,conductor,mujer,no,12,6,99.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,20.2,13.1,10,0


### Validation Framework

In [978]:
from sklearn.model_selection import train_test_split

In [980]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [982]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_full_train = df_full_train.reset_index(drop=True)

In [984]:
y_train = df_train.clasacc.values
y_val = df_val.clasacc.values
y_test = df_test.clasacc.values

In [986]:
del df_train["clasacc"]
del df_val["clasacc"]
del df_test["clasacc"]

In [988]:
df_train.head()

,accident_type,nombre_de_asentamiento,nombre_de_la_vialidad,causaacci,sex,alcohol,month,weekday,id_edad,casualties_kil,casualties_inj,heavy_vehicles,light_vehicles,two_wheelers,other_vehicles,precip,windgust,visibility,severerisk
0,crucero,centro,tapia,conductor,hombre,no,8,4,63.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,50.4,11.6,30
1,crucero,maria_luisa,padre_mier,conductor,hombre,no,2,2,20.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,50.4,14.4,10
2,colisión_con_motocicleta,las_cumbres,leones,conductor,hombre,no,6,6,99.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,31.7,11.6,30
3,lateral,valle_del_mirador,paseo_llavale,conductor,hombre,no,9,4,99.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,36.0,14.4,10
4,colisión_con_motocicleta,fomerrey_116,nepenta,conductor,hombre,no,9,4,24.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,36.0,14.4,10


### Decision Tree Classifier

In [1036]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score

In [1038]:
train_dicts = df_train.to_dict(orient="records")

In [1040]:
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)

In [1042]:
dt = DecisionTreeClassifier(max_depth=5, random_state=42)
dt.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, random_state=42)

#### Evaluate Model

In [1049]:
val_dicts = df_val.to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [1051]:
from sklearn.metrics import classification_report, confusion_matrix

# Make predictions on the validation set
y_pred = dt.predict(X_val)

print("Validation Set Performance:")
print(classification_report(y_val, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

Validation Set Performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5563
           1       1.00      1.00      1.00       269
           2       1.00      1.00      1.00        12

    accuracy                           1.00      5844
   macro avg       1.00      1.00      1.00      5844
weighted avg       1.00      1.00      1.00      5844

Confusion Matrix:
[[5563    0    0]
 [   0  269    0]
 [   0    0   12]]


#### Visualize Tree

In [1054]:
from sklearn.tree import export_text

In [1056]:
print(export_text(dt, feature_names=dv.get_feature_names_out()))

|--- casualties_inj <= 0.50
|   |--- casualties_kil <= 0.50
|   |   |--- class: 0
|   |--- casualties_kil >  0.50
|   |   |--- class: 2
|--- casualties_inj >  0.50
|   |--- casualties_kil <= 0.50
|   |   |--- class: 1
|   |--- casualties_kil >  0.50
|   |   |--- class: 2

